In [24]:
# Import required libraries
import pandas as pd
pd.set_option('display.max_columns', None) # pandas show all columns of table instead of restricted
pd.options.display.max_rows
import numpy as np 
import matplotlib.pyplot as plt
import plotly.express as px

from eda_quality import correction as correct
from eda_quality import manipulation as manipulate
from eda_quality import inspection as inspect

from envirocar import TrackAPI, DownloadClient, BboxSelector, ECConfig

# create an initial but optional config and an api client
config = ECConfig()
track_api = TrackAPI(api_client=DownloadClient(config=config))

In [25]:
bbox = BboxSelector([
    7.554130554199218, # min_x
    51.95590322041212, # min_y
    7.590351104736328, # max_x
    51.97874790276371  # max_y
])
# issue a query
df_tracks = track_api.get_tracks(bbox=bbox, num_results=40)

In [26]:
inspect.count_tracks(df_tracks)

40


Get a first glance of the dublicates by showing the dublicated data

In [27]:
dublicates = inspect.show_dublicated_tracks(df_tracks)
dublicates.head()

Dublicated tracks: ['5ddfd2ba44ea85025c02d828', '5ddfd2bc44ea85025c02dbce']


,id,time,geometry,Engine Load.value,Engine Load.unit,Calculated MAF.value,Calculated MAF.unit,Speed.value,Speed.unit,CO2.value,CO2.unit,Intake Pressure.value,Intake Pressure.unit,Rpm.value,Rpm.unit,Intake Temperature.value,Intake Temperature.unit,Consumption (GPS-based).value,Consumption (GPS-based).unit,GPS Altitude.value,GPS Altitude.unit,Throttle Position.value,Throttle Position.unit,GPS Bearing.value,GPS Bearing.unit,Consumption.value,Consumption.unit,GPS Accuracy.value,GPS Accuracy.unit,CO2 Emission (GPS-based).value,CO2 Emission (GPS-based).unit,GPS Speed.value,GPS Speed.unit,track.id,track.length,track.begin,track.end,sensor.type,sensor.engineDisplacement,sensor.model,sensor.id,sensor.fuelType,sensor.constructionYear,sensor.manufacturer,track.appVersion,track.touVersion,GPS HDOP.value,GPS HDOP.unit,GPS PDOP.value,GPS PDOP.unit,GPS VDOP.value,GPS VDOP.unit,MAF.value,MAF.unit,O2 Lambda Voltage ER.value,O2 Lambda Voltage ER.unit,O2 Lambda Voltage.value,O2 Lambda Voltage.unit
0,5ddfd2ba44ea85025c02d82a,2019-11-28T13:41:36+00:00,POINT (7.65013 51.94115),441.321860,%,346.627217,g/s,-9.206349,km/h,267.768457,kg/h,502.171124,kPa,4784.997272,u/min,17.47449,c,NaN,NaN,100.692023,m,23.988255,%,223.737910,deg,113.944024,l/h,9.891432,%,NaN,NaN,0.000000,km/h,5ddfd2ba44ea85025c02d828,9.220333,2019-11-28T13:41:36Z,2019-11-28T13:57:24Z,car,1699,A 170,559e22c2e4b07207d8977998,gasoline,2004,Mercedes Benz,NaN,NaN,0.900000,precision,1.500000,precision,1.246763,precision,NaN,NaN,NaN,NaN,NaN,NaN
1,5ddfd2ba44ea85025c02d82c,2019-11-28T13:41:47+00:00,POINT (7.64931 51.94163),61.418813,%,15.605188,g/s,49.911602,km/h,12.054959,kg/h,60.924582,kPa,1717.722820,u/min,8.00000,c,NaN,NaN,100.738323,m,19.411429,%,315.329161,deg,5.129770,l/h,8.000000,%,NaN,NaN,46.184018,km/h,5ddfd2ba44ea85025c02d828,9.220333,2019-11-28T13:41:36Z,2019-11-28T13:57:24Z,car,1699,A 170,559e22c2e4b07207d8977998,gasoline,2004,Mercedes Benz,NaN,NaN,0.829757,precision,1.389271,precision,1.189271,precision,NaN,NaN,NaN,NaN,NaN,NaN
2,5ddfd2ba44ea85025c02d82d,2019-11-28T13:41:52+00:00,POINT (7.64866 51.94210),99.426585,%,25.954246,g/s,52.134077,km/h,20.049575,kg/h,95.363385,kPa,1831.662189,u/min,9.00000,c,NaN,NaN,101.438228,m,45.000001,%,322.014955,deg,8.531734,l/h,10.864922,%,NaN,NaN,50.529060,km/h,5ddfd2ba44ea85025c02d828,9.220333,2019-11-28T13:41:36Z,2019-11-28T13:57:24Z,car,1699,A 170,559e22c2e4b07207d8977998,gasoline,2004,Mercedes Benz,NaN,NaN,0.800000,precision,1.400000,precision,1.200000,precision,NaN,NaN,NaN,NaN,NaN,NaN
3,5ddfd2ba44ea85025c02d82e,2019-11-28T13:41:57+00:00,POINT (7.64804 51.94263),35.056864,%,4.091531,g/s,43.470901,km/h,3.160695,kg/h,33.553191,kPa,820.673855,u/min,9.00000,c,NaN,NaN,101.282162,m,13.779412,%,324.795254,deg,1.344977,l/h,8.000000,%,NaN,NaN,43.041294,km/h,5ddfd2ba44ea85025c02d828,9.220333,2019-11-28T13:41:36Z,2019-11-28T13:57:24Z,car,1699,A 170,559e22c2e4b07207d8977998,gasoline,2004,Mercedes Benz,NaN,NaN,0.900000,precision,1.500000,precision,1.200000,precision,NaN,NaN,NaN,NaN,NaN,NaN
4,5ddfd2ba44ea85025c02d82f,2019-11-28T13:42:02+00:00,POINT (7.64777 51.94289),31.904417,%,3.459813,g/s,6.399497,km/h,2.672695,kg/h,30.000001,kPa,776.157749,u/min,9.00000,c,NaN,NaN,99.117048,m,13.583942,%,327.392114,deg,1.137317,l/h,8.000000,%,NaN,NaN,6.598425,km/h,5ddfd2ba44ea85025c02d828,9.220333,2019-11-28T13:41:36Z,2019-11-28T13:57:24Z,car,1699,A 170,559e22c2e4b07207d8977998,gasoline,2004,Mercedes Benz,NaN,NaN,0.888122,precision,1.488122,precision,1.200000,precision,NaN,NaN,NaN,NaN,NaN,NaN


If the above returns a list with minimal two ids, you may want to inspect the dublicated tracks by eye.    
To plot the dublicated tracks, first have a look which variable in each track has at least some values.    
For this inspect the proportion of missing values per track and select a variable you like to plot.  
In general the envirocar variable 'Speed.value' seems to be appropriate in most cases, as it usually provides valid values.

In [28]:
count, proportion = inspect.missing_values_per_track(dublicates)
proportion

,id,time,geometry,Engine Load.value,Engine Load.unit,Calculated MAF.value,Calculated MAF.unit,Speed.value,Speed.unit,CO2.value,CO2.unit,Intake Pressure.value,Intake Pressure.unit,Rpm.value,Rpm.unit,Intake Temperature.value,Intake Temperature.unit,Consumption (GPS-based).value,Consumption (GPS-based).unit,GPS Altitude.value,GPS Altitude.unit,Throttle Position.value,Throttle Position.unit,GPS Bearing.value,GPS Bearing.unit,Consumption.value,Consumption.unit,GPS Accuracy.value,GPS Accuracy.unit,CO2 Emission (GPS-based).value,CO2 Emission (GPS-based).unit,GPS Speed.value,GPS Speed.unit,track.id,track.length,track.begin,track.end,sensor.type,sensor.engineDisplacement,sensor.model,sensor.id,sensor.fuelType,sensor.constructionYear,sensor.manufacturer,track.appVersion,track.touVersion,GPS HDOP.value,GPS HDOP.unit,GPS PDOP.value,GPS PDOP.unit,GPS VDOP.value,GPS VDOP.unit,MAF.value,MAF.unit,O2 Lambda Voltage ER.value,O2 Lambda Voltage ER.unit,O2 Lambda Voltage.value,O2 Lambda Voltage.unit
track.id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
5ddfd2ba44ea85025c02d828,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,100.0,100.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,100.0,100.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,100.0,100.0,0.0,0.0,0.0,0.0,0.0,0.0,100.0,100.0,100.0,100.0,100.0,100.0
5ddfd2bc44ea85025c02dbce,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,100.0,100.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,100.0,100.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,100.0,100.0,0.0,0.0,0.0,0.0,0.0,0.0,100.0,100.0,100.0,100.0,100.0,100.0


If you have selected a variable to plot ...  do some plotting!     
In the legend of the plot click on the tracks to discover the dublicates.  

In [29]:
inspect.plot_tracks(dublicates, 'Speed.value')

Create a new dataframe by dropping dublicates from the current dataframe.

In [30]:
uniqueTracks=correct.drop_dublicates(df_tracks)

Deleted rows:  186


Check how many tracks are now in the dataframe

In [31]:
inspect.count_tracks(uniqueTracks)

39
